# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [28]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [29]:
# checking the current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [30]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Check total number of rows 
print(len(full_data_rows_list))
# Check the data row list
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [31]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [32]:
# Create a  connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [33]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [34]:
session.set_keyspace('sparkify')

## Queries that ask questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4

In [50]:
session.execute("""CREATE TABLE IF NOT EXISTS session_songs
(sessionId int, 
itemInSession int, 
artist text, 
song text, 
length float,
PRIMARY KEY(sessionId, itemInSession))""")              

In [51]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist, name, gender, itemInSession, last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist, song, float(length)))

##### The SELECT statement queries for artist, song and length for a given sessionId (here 338) and a given itemInSession (here 4).

In [57]:
# Add in the SELECT statement to verify the data was entered into the table

query = ("""SELECT artist, song, length 
FROM session_songs 
WHERE sessionId = 338 AND itemInSession = 4""")

try:
    rows = session.execute(query)
except Exception as e:
    print(e)  
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.artist, row.song, row.length])
print(t)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [58]:
session.execute("""CREATE TABLE IF NOT EXISTS event_log
(user_id int,
session_id int,
item_in_session int,
artist text, 
song text, 
first_name text, 
last_name text,
PRIMARY KEY ((user_id, session_id), item_in_session))""")         

In [59]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO event_log (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))  

##### The SELECT statement queries for artist, song a and user's first and last name for a given userId and sessionId. The results are ordered descending by itemInSession.

In [60]:
# Add in the SELECT statement to verify the data was entered into the table

query = "SELECT artist, song, first_name, last_name FROM event_log \
            WHERE user_id = 10 and session_id = 182 \
            ORDER by item_in_session DESC"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)  
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'First Name', 'Last Name'])
for row in rows:
    t.add_row([row.artist, row.song, row.first_name, row.last_name])
print(t)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [61]:
session.execute("""CREATE TABLE IF NOT EXISTS song_users 
(song text, 
user_id int, 
first_name text, 
last_name text, 
PRIMARY KEY (song, user_id))""")            

In [62]:
file = 'event_datafile_new.csv'

df = pd.read_csv(file, usecols=[1, 4, 9, 10])
df.drop_duplicates(inplace=True)

for ix, row in df.iterrows():
    query = "INSERT INTO song_users (song, user_id, first_name, last_name)"
    query = query + " VALUES (%s, %s, %s, %s)"
    session.execute(query, (row['song'], row['userId'], row['firstName'], row['lastName']))

##### The SELECT statement queries for user's first and last name for a given song (here All Hands Against His Own).

In [63]:
# Add in the SELECT statement to verify the data was entered into the table

query = "SELECT first_name, last_name FROM song_users \
         WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e) 
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['First Name', 'Last Name'])
for row in rows:
    t.add_row([row.first_name, row.last_name])
print(t)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [64]:
session.execute("drop table session_songs")
session.execute("drop table event_log")
session.execute("drop table song_users")

### Close the session and cluster connection¶

In [65]:
session.shutdown()
cluster.shutdown()